In [1]:
import pandas as pd

import sys
sys.path.append("/home/paolo/airflow/dags")

from custom_code import custom_lib as cl
import re

fp = cl.path_utils.get_today_remotive()
print(fp)

df = pd.read_json(fp)
print("Columns :", df.columns)

df.head()
print(df.category.unique())


sw_df = df[df["category"] == "Software Development"]
sw_df.head()
sw_df.candidate_required_location.unique()
sw_df["location"] = sw_df.candidate_required_location.str.lower()

weg = sw_df[
    (sw_df["location"].str.contains("europe"))
    | (sw_df["location"].str.contains("germany"))
    | (sw_df["location"].str.contains("worldwide"))
]

print(weg.salary.unique())
currency_api = cl.currency_api.CachedCurrencyAPI()
t = currency_api.convert_usd_to_euros(50.000)

def salary_cleaner(s: str):
    out = s.lower()
    number_k = out.count("k")

    is_hourly = False
    is_monthly = False
    is_dollar = False
    is_euro = False

    if number_k >= 1:
        out = out.replace("k", "000")

    if "hour" in out:
        is_hourly = True
    if "month" in out:
        is_monthly = True

    if "$" in out or "dollar" in out:
        is_dollar = True

    if "€" in out or "euro" in out:
        is_euro = True

    if is_hourly and is_monthly:
        raise Exception(f"Is it hourly on monthly salary?? {s}") 

    if is_dollar and is_euro:
        raise Exception(f"Is it in dollar or euro? {s}")

    numbers = re.findall("\d+", out)
    salary = -1
    if len(numbers) == 0:
        return float(salary)
    elif len(numbers) == 1:
        salary = float(numbers[0])
    # If it's a range, take average
    elif len(numbers) == 2:
        salary = (float(numbers[0]) + float(numbers[1])) / 2
    else:
        raise Exception(f"Unhandled case, 3 or more salary numbers? {s}")
    if is_dollar:
        salary_in_euros = currency_api.convert_usd_to_euros(salary)
    # Else assume it's euros
    else:
        salary_in_euros = salary
    
    if is_monthly:
        annual_salary_in_euros = salary_in_euros * 12
    elif is_hourly:
        # 8 hours / day, 20 days / month, 12 months / year
        annual_salary_in_euros = salary_in_euros * 8 * 20 * 12
    else:
        annual_salary_in_euros = salary_in_euros
    return annual_salary_in_euros

weg["salary"] = weg["salary"].apply(salary_cleaner)


print(weg.salary.unique())

weg.head()

/data/remotive/2023-01-25_remotive.json
Columns : Index(['id', 'url', 'title', 'company_name', 'company_logo', 'category',
       'tags', 'job_type', 'publication_date', 'candidate_required_location',
       'salary', 'description', 'company_logo_url'],
      dtype='object')
['QA' 'Finance / Legal' 'Business' 'Design' 'Customer Service'
 'Software Development' 'Writing' 'Sales' 'Marketing' 'All others'
 'Human Resources' 'DevOps / Sysadmin' 'Product' 'Data']
['' '€63k' '$50k-$70k' '$125k - $200k' '$60k-$108k' '$50K - $60K'
 '$80k-$180k' '$50k-$70K' '$60k - $80k' '$40K - $80K' '$30 - $50k'
 '$48k-$96k' '€5495 /month' '$110-190 hourly' '$65k-$80k' '€6k/month']
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON from FS
Cache hit, returning JSON f

/tmp/ipykernel_8468/3513619667.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sw_df["location"] = sw_df.candidate_required_location.str.lower()
/tmp/ipykernel_8468/3513619667.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weg["salary"] = weg["salary"].apply(salary_cleaner)


,id,url,title,company_name,company_logo,category,tags,job_type,publication_date,candidate_required_location,salary,description,company_logo_url,location
7,1564286,https://remotive.com/remote-jobs/software-dev/...,PHP & JavaScript Fullstack Developer (m/f/d),Social Sweethearts,https://remotive.com/job/1564286/logo,Software Development,"[api, big data, cassandra, developer, devops, ...",full_time,2023-01-24T05:39:11,Germany,-1.0,<p>We are a digital media and games company. O...,NaN,germany
39,1564926,https://remotive.com/remote-jobs/software-dev/...,Android Developer,SD Solutions,https://remotive.com/job/1564926/logo,Software Development,"[amazon, android, developer, java, machine lea...",full_time,2023-01-23T03:39:33,Worldwide,-1.0,"<p>For our new client, SD Solutions is looking...",NaN,worldwide
80,1565263,https://remotive.com/remote-jobs/software-dev/...,Front-end Developer ( React/Angular ),Keeper Solutions,https://remotive.com/job/1565263/logo,Software Development,"[big data, CSS, developer, html, javascript, j...",full_time,2023-01-21T13:39:53,Europe,-1.0,<p><br>Keeper Solutions is a market-leading so...,NaN,europe
117,1557608,https://remotive.com/remote-jobs/software-dev/...,Senior Full-Stack Engineer,Passionfroot,https://remotive.com/job/1557608/logo,Software Development,"[devops, javascript, postgresql, react, growth...",full_time,2023-01-20T11:40:08,Germany,-1.0,"<div class=""h3"">🔴 About Passionfroot</div>\n<p...",NaN,germany
179,1501053,https://remotive.com/remote-jobs/software-dev/...,Frontend Developer (JavaScript + React),Toggl Hire,https://remotive.com/job/1501053/logo,Software Development,"[AWS, backend, developer, frontend, golang, gr...",full_time,2023-01-18T10:30:38,"Europe, European timezones",63000.0,<p>We are looking for a Frontend Developer to ...,https://remotive.com/job/1501053/logo,"europe, european timezones"
